In [1]:
import os
import sys
sys.path.append('../../detr')
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from scipy import ndimage
import bisect
from src.eval import infer_visualize
from src.main import get_model
import json
import argparse
import torch.nn as nn
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [7]:
def get_args():
    parser = argparse.ArgumentParser()

    parser.add_argument('--data_root_dir',
                        # required=True,
                        help="Root data directory for images and labels")
    parser.add_argument('--config_file',
                        # required=True,
                        help="Filepath to the config containing the args")
    parser.add_argument('--backbone',
                        default='resnet18',
                        help="Backbone for the model")
    parser.add_argument(
        '--data_type',
        choices=['detection', 'structure'],
        default='structure',
        help="toggle between structure recognition and table detection")
    parser.add_argument('--model_load_path', help="The path to trained model")
    parser.add_argument('--load_weights_only', action='store_true')
    parser.add_argument('--model_save_dir', help="The output directory for saving model params and checkpoints")
    parser.add_argument('--metrics_save_filepath',
                        help='Filepath to save grits outputs',
                        default='')
    parser.add_argument('--debug_save_dir',
                        help='Filepath to save visualizations',
                        default='debug')
    parser.add_argument('--table_words_dir',
                        help="Folder containg the bboxes of table words")
    parser.add_argument('--mode',
                        choices=['train', 'eval'],
                        default='train',
                        help="Modes: training (train) and evaluation (eval)")
    parser.add_argument('--debug', action='store_true')
    parser.add_argument('--device')
    parser.add_argument('--lr', type=float)
    parser.add_argument('--lr_drop', type=int)
    parser.add_argument('--lr_gamma', type=float)
    parser.add_argument('--epochs', type=int)
    parser.add_argument('--checkpoint_freq', default=1, type=int)
    parser.add_argument('--batch_size', type=int)
    parser.add_argument('--num_workers', type=int)
    parser.add_argument('--train_max_size', type=int)
    parser.add_argument('--val_max_size', type=int)
    parser.add_argument('--test_max_size', type=int)
    parser.add_argument('--eval_pool_size', type=int, default=1)
    parser.add_argument('--eval_step', type=int, default=1)
    
    parser.add_argument('--overlap', action='store_true')
    parser.add_argument("--overlap_loss_coef", type=int, default=1)
    args, unknown = parser.parse_known_args()
    return args

In [3]:
image_folder = '/home/suqi/dataset/pdf_images_filter_1'

# 随机选择部分样本进行测试
k = 500
np.random.seed(0)
# test_list = sorted([name for name in os.listdir(image_folder) if 'COL' in name])
test_list = sorted([name for name in os.listdir(image_folder)])
# np.random.shuffle(test_list)
# test_list = test_list[:k]
img_paths = [os.path.join(image_folder, name) for name in test_list]

# img_names = ['_'.join(name.split('_')[:-1]) + '.jpg' for name in os.listdir('/home/suqi/dataset/temp/visualize')]
# img_paths = [os.path.join(image_folder, name) for name in list(set(img_names))]
print(len(img_paths))

42


In [4]:
# load images
samples = []
img_paths_filter = []
errors = 0
for path in img_paths:
    try:
        samples.append(Image.open(path).convert('RGB'))
        img_paths_filter.append(path)
    except Exception as e:
        errors += 1
        print(f"Exception when load image {path}")
img_paths = img_paths_filter
print(errors)

0


In [8]:
args = get_args()
args.config_file = "../structure_config.json"
args.data_type = None
args.model_load_path = "/home/suqi/model/TATR/finetune/train_finetune_resnet34/model_best.pth"
args.backbone = "resnet34"
args.debug_save_dir = "/home/suqi/dataset/temp/visualize_filter_pdf_v5/"
os.makedirs(args.debug_save_dir, exist_ok=True)
args.device = "cuda:4"

cmd_args = args.__dict__
config_args = json.load(open(cmd_args['config_file'], 'rb'))
for key, value in cmd_args.items():
    if not key in config_args or not value is None:
        config_args[key] = value
# config_args.update(cmd_args)
args = type('Args', (object,), config_args)

# define and load model
device = torch.device(args.device)
model, _, _ = get_model(args, device)

loading model from checkpoint
load model parameters successfully!


In [9]:
# 约3.00s/it
infer_visualize(model, samples, device, img_paths, args.debug_save_dir, True)

Inferring: : 42it [02:54,  4.16s/it]


In [11]:
del model
torch.cuda.empty_cache()

NameError: name 'model' is not defined